In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from more_itertools import padded
import sys
import math

from IPython.display import Markdown as md, SVG

In [2]:
names = []
nameasc = []
genders = []
probs = []
maxlen = 0

f = open("name_gender.csv")
data = f.readlines()
f.close()
for line in data:
    name, gender, prob = line.split(',')
    names.append(name.lower())
    if name.__len__() > maxlen:
        maxlen = name.__len__()
    nameasc.append([ord(ch) for ch in name.lower()])
    genders.append([1,0] if gender == 'M' else [0,1])
    probs.append(np.float16(prob.strip("\n")))

In [3]:
del name, data, gender, prob
names = np.array(names)
genders = np.array(genders, dtype=np.int8)
probs = np.array(probs, dtype=np.float16)

In [42]:
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Input
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import model_to_dot
import pydot as pyd
keras.utils.vis_utils.pydot = pyd

## Word 2 vec Features

https://github.com/minimaxir/char-embeddings/blob/master/glove.840B.300d-char.txt

In [5]:
f = open("assets\\vecRep\\glove.840B.300d-char.txt")
data = f.readlines()
f.close()
glove = {}
for line in data:
    lineTmp = line.split()
    glove[lineTmp[0]] = np.array(lineTmp[1:], dtype=np.float16)

<b> Now all we need to do is replace the alphabets with their representations </b>

In [6]:
for idx in range(nameasc.__len__()):
    nameasc[idx] = ( maxlen - nameasc[idx].__len__() ) * [0] + [chr(asc) for asc in nameasc[idx]]

<b>So if we use this as is, we have a sequence of 15 representations for the name, each representation in a 300 dim space</b><br/>
We must reduce it

## Autoencoder

In [7]:
gloveEmbed300 = np.array(list(glove.values()))
gloveEmbed300

array([[ 0.1443 , -0.4607 ,  0.2051 , ...,  0.1782 , -0.0158 , -0.3984 ],
       [-0.04562,  0.0507 ,  0.184  , ..., -0.1482 , -0.03186,  0.1567 ],
       [ 0.0838 , -0.472  ,  0.2108 , ...,  0.2568 ,  0.06213, -0.25   ],
       ...,
       [ 0.239  , -0.2869 ,  0.11664, ...,  0.1824 , -0.07245, -0.1365 ],
       [ 0.2397 , -0.2725 ,  0.09875, ...,  0.1975 , -0.0751 , -0.1136 ],
       [ 0.10345,  0.06177, -0.3564 , ..., -0.06207, -0.5938 , -0.1527 ]],
      dtype=float16)

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler = MinMaxScaler()
X = scaler.fit_transform(gloveEmbed300)
X

array([[0.401  , 0.1416 , 0.6685 , ..., 0.639  , 0.762  , 0.     ],
       [0.0418 , 0.9824 , 0.6436 , ..., 0.     , 0.741  , 0.949  ],
       [0.2866 , 0.12354, 0.6753 , ..., 0.793  , 0.865  , 0.2537 ],
       ...,
       [0.58   , 0.4275 , 0.563  , ..., 0.6475 , 0.6875 , 0.4478 ],
       [0.5815 , 0.4512 , 0.542  , ..., 0.677  , 0.684  , 0.4868 ],
       [0.3237 , 1.     , 0.     , ..., 0.1687 , 0.     , 0.42   ]],
      dtype=float16)

In [10]:
inp = Input(shape=(300,))
encoded = Dense(25, activation='relu')(inp)
out = Dense(300, activation='sigmoid')(encoded)

In [11]:
autoencoder = Model(inputs=inp, outputs=out)
autoencoder.compile(optimizer='adam', loss='mse')

## Test Val Split

In [12]:
np.random.seed(42)
vald_indices = np.random.choice(range(X.shape[0]), math.ceil(0.15*X.shape[0]), replace=False)
X_vald = X[vald_indices]

In [13]:
mask = np.ones(X.shape[0], dtype=bool)
mask[vald_indices] = False
X_train = X[mask]

In [14]:
autoencoder.fit(X_train, X_train, batch_size=50, epochs=1000, verbose=1, validation_data=(X_vald, X_vald))

Train on 79 samples, validate on 15 samples
Epoch 1/1000
79/79 [==============================] - 0s 6ms/step - loss: 0.0462 - val_loss: 0.0435
Epoch 2/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0446 - val_loss: 0.0424
Epoch 3/1000
79/79 [==============================] - 0s 165us/step - loss: 0.0436 - val_loss: 0.0416
Epoch 4/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0429 - val_loss: 0.0408
Epoch 5/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0422 - val_loss: 0.0400
Epoch 6/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0415 - val_loss: 0.0392
Epoch 7/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0407 - val_loss: 0.0383
Epoch 8/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0399 - val_loss: 0.0374
Epoch 9/1000
79/79 [==============================] - 0s 215us/step - loss: 0.0389 - val_loss: 0.0364
Epoch 10/1000
79/79 [===================

79/79 [==============================] - 0s 139us/step - loss: 0.0136 - val_loss: 0.0146
Epoch 81/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0135 - val_loss: 0.0146
Epoch 82/1000
79/79 [==============================] - 0s 178us/step - loss: 0.0134 - val_loss: 0.0145
Epoch 83/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0133 - val_loss: 0.0144
Epoch 84/1000
79/79 [==============================] - 0s 158us/step - loss: 0.0132 - val_loss: 0.0143
Epoch 85/1000
79/79 [==============================] - 0s 190us/step - loss: 0.0130 - val_loss: 0.0143
Epoch 86/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0129 - val_loss: 0.0142
Epoch 87/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0129 - val_loss: 0.0141
Epoch 88/1000
79/79 [==============================] - 0s 203us/step - loss: 0.0128 - val_loss: 0.0141
Epoch 89/1000
79/79 [==============================] - 0s 164us/step - loss: 0.0127 - v

79/79 [==============================] - 0s 177us/step - loss: 0.0090 - val_loss: 0.0105
Epoch 159/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0090 - val_loss: 0.0105
Epoch 160/1000
79/79 [==============================] - 0s 172us/step - loss: 0.0090 - val_loss: 0.0104
Epoch 161/1000
79/79 [==============================] - 0s 202us/step - loss: 0.0090 - val_loss: 0.0104
Epoch 162/1000
79/79 [==============================] - 0s 253us/step - loss: 0.0089 - val_loss: 0.0104
Epoch 163/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0089 - val_loss: 0.0103
Epoch 164/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0089 - val_loss: 0.0103
Epoch 165/1000
79/79 [==============================] - ETA: 0s - loss: 0.010 - 0s 127us/step - loss: 0.0089 - val_loss: 0.0103
Epoch 166/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0089 - val_loss: 0.0103
Epoch 167/1000
79/79 [==============================] -

79/79 [==============================] - 0s 127us/step - loss: 0.0076 - val_loss: 0.0095
Epoch 237/1000
79/79 [==============================] - 0s 114us/step - loss: 0.0075 - val_loss: 0.0095
Epoch 238/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0075 - val_loss: 0.0094
Epoch 239/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0075 - val_loss: 0.0095
Epoch 240/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0075 - val_loss: 0.0094
Epoch 241/1000
79/79 [==============================] - 0s 164us/step - loss: 0.0075 - val_loss: 0.0094
Epoch 242/1000
79/79 [==============================] - 0s 127us/step - loss: 0.0075 - val_loss: 0.0094
Epoch 243/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0074 - val_loss: 0.0094
Epoch 244/1000
79/79 [==============================] - 0s 126us/step - loss: 0.0074 - val_loss: 0.0094
Epoch 245/1000
79/79 [==============================] - 0s 139us/step - loss: 0

79/79 [==============================] - 0s 139us/step - loss: 0.0064 - val_loss: 0.0090
Epoch 315/1000
79/79 [==============================] - 0s 146us/step - loss: 0.0064 - val_loss: 0.0091
Epoch 316/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0064 - val_loss: 0.0091
Epoch 317/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0064 - val_loss: 0.0091
Epoch 318/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0064 - val_loss: 0.0091
Epoch 319/1000
79/79 [==============================] - 0s 127us/step - loss: 0.0064 - val_loss: 0.0091
Epoch 320/1000
79/79 [==============================] - 0s 151us/step - loss: 0.0064 - val_loss: 0.0091
Epoch 321/1000
79/79 [==============================] - 0s 127us/step - loss: 0.0064 - val_loss: 0.0090
Epoch 322/1000
79/79 [==============================] - ETA: 0s - loss: 0.007 - 0s 139us/step - loss: 0.0063 - val_loss: 0.0090
Epoch 323/1000
79/79 [==============================] -

...

79/79 [==============================] - 0s 196us/step - loss: 0.0043 - val_loss: 0.0083
Epoch 628/1000
79/79 [==============================] - 0s 165us/step - loss: 0.0043 - val_loss: 0.0083
Epoch 629/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0043 - val_loss: 0.0083
Epoch 630/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0043 - val_loss: 0.0083
Epoch 631/1000
79/79 [==============================] - 0s 158us/step - loss: 0.0042 - val_loss: 0.0083
Epoch 632/1000
79/79 [==============================] - 0s 164us/step - loss: 0.0042 - val_loss: 0.0083
Epoch 633/1000
79/79 [==============================] - 0s 190us/step - loss: 0.0042 - val_loss: 0.0083
Epoch 634/1000
79/79 [==============================] - 0s 146us/step - loss: 0.0042 - val_loss: 0.0083
Epoch 635/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0042 - val_loss: 0.0083
Epoch 636/1000
79/79 [==============================] - 0s 152us/step - loss: 0

Epoch 706/1000
79/79 [==============================] - 0s 146us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 707/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 708/1000
79/79 [==============================] - ETA: 0s - loss: 0.005 - 0s 177us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 709/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 710/1000
79/79 [==============================] - 0s 202us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 711/1000
79/79 [==============================] - 0s 215us/step - loss: 0.0041 - val_loss: 0.0082
Epoch 712/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 713/1000
79/79 [==============================] - 0s 127us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 714/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0041 - val_loss: 0.0083
Epoch 715/1000
79/79 [==================

79/79 [==============================] - 0s 178us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 785/1000
79/79 [==============================] - 0s 151us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 786/1000
79/79 [==============================] - 0s 153us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 787/1000
79/79 [==============================] - 0s 164us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 788/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 789/1000
79/79 [==============================] - 0s 265us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 790/1000
79/79 [==============================] - 0s 146us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 791/1000
79/79 [==============================] - 0s 202us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 792/1000
79/79 [==============================] - 0s 127us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 793/1000
79/79 [==============================] - 0s 152us/step - loss: 0

79/79 [==============================] - 0s 152us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 863/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 864/1000
79/79 [==============================] - 0s 152us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 865/1000
79/79 [==============================] - 0s 215us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 866/1000
79/79 [==============================] - 0s 184us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 867/1000
79/79 [==============================] - 0s 184us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 868/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 869/1000
79/79 [==============================] - 0s 178us/step - loss: 0.0040 - val_loss: 0.0083
Epoch 870/1000
79/79 [==============================] - 0s 202us/step - loss: 0.0040 - val_loss: 0.0082
Epoch 871/1000
79/79 [==============================] - 0s 354us/step - loss: 0

79/79 [==============================] - 0s 203us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 941/1000
79/79 [==============================] - 0s 203us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 942/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 943/1000
79/79 [==============================] - 0s 165us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 944/1000
79/79 [==============================] - 0s 165us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 945/1000
79/79 [==============================] - 0s 139us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 946/1000
79/79 [==============================] - 0s 164us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 947/1000
79/79 [==============================] - 0s 177us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 948/1000
79/79 [==============================] - 0s 127us/step - loss: 0.0039 - val_loss: 0.0083
Epoch 949/1000
79/79 [==============================] - 0s 151us/step - loss: 0

In [27]:
from sklearn.metrics import mean_squared_error
X_reconstr = autoencoder.predict(X)

In [28]:
print(f"Mean Squared Error: {mean_squared_error(X, X_reconstr)}")

Mean Squared Error: 0.004595634061843157


In [29]:
encoder = Model(inp, encoded)
X_encoded = encoder.predict(X)
X_encoded

array([[ 0.       ,  5.948668 ,  0.       , ...,  0.       ,  8.546268 ,
         4.5678515],
       [ 0.       ,  6.0190344,  0.       , ...,  0.       ,  8.722171 ,
         0.       ],
       [ 0.       ,  6.957459 ,  0.       , ...,  0.       ,  8.752112 ,
         3.4407334],
       ...,
       [ 0.       ,  4.473628 ,  0.       , ...,  0.       ,  7.7107444,
         5.232554 ],
       [ 0.       ,  4.3428826,  0.       , ...,  0.       ,  7.849421 ,
         5.3157773],
       [ 0.       ,  0.7392084,  0.       , ...,  0.       ,  0.       ,
        11.52013  ]], dtype=float32)

In [30]:
X_encoded.shape

(94, 25)

In [31]:
for idx,key in enumerate(glove.keys()):
    glove[key] = X_encoded[idx]

In [33]:
glove['a']

array([0.       , 4.472513 , 0.       , 0.       , 2.1842103, 0.       ,
       0.       , 0.       , 4.6267138, 7.228192 , 0.       , 0.       ,
       6.047776 , 0.       , 2.3355665, 0.       , 5.715703 , 6.5474143,
       0.       , 5.2858877, 6.575798 , 6.4386816, 0.       , 7.7341027,
       5.2449417], dtype=float32)

## Features

In [35]:
tensorasc = []
for ascname in nameasc:
    tensornm = []
    for num in ascname:
        if num in glove.keys():
            tensornm.append(glove[num])
        else:
            tensornm.append([0]*25)
    tensorasc.append(tensornm)
tensorasc = np.array(tensorasc)

In [36]:
tensorasc[0][9:]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 4.4725132 , 0.        , 0.        , 2.1842103 ,
        0.        , 0.        , 0.        , 4.62671375, 7.22819185,
        0.        , 0.        , 6.04777622, 0.        , 2.33556652,
        0.        , 5.71570301, 6.5474143 , 0.        , 5.28588772,
        6.57579803, 6.4386816 , 0.        , 7.73410273, 5.24494171],
       [0.        , 4.4725132 , 0.        , 0.        , 2.1842103 ,
        0.        , 0.        , 0.        , 4.62671375, 7.22819185,
        0.        , 0.        , 6.04777622, 0.        , 2.33556652,
        0.        , 5.71570301, 6.5474143 , 0.        , 5.28588772,
        6.57579803, 6.4386816 , 0.        , 7.

## Test Train Split

In [37]:
np.random.seed(42)
test_indices = np.random.choice(range(names.__len__()), math.ceil(0.3*names.__len__()), replace=False)
vald_indices = np.random.choice(test_indices, math.ceil(0.5*test_indices.__len__()), replace=False)
X_test = tensorasc[test_indices]
Y_test = genders[test_indices]
X_vald = tensorasc[vald_indices]
Y_vald = genders[vald_indices]
train_indices = np.ones(names.__len__(), dtype=bool)
train_indices[test_indices] = False
X_train = tensorasc[train_indices]
Y_train = genders[train_indices]

In [39]:
X_train.shape, X_test.shape,X_vald.shape

((66517, 15, 25), (28508, 15, 25), (14254, 15, 25))

## Model

In [40]:
modelGl = Sequential()
modelGl.add(LSTM(250, return_sequences=False, input_shape=(15, 25)))
modelGl.add(Dropout(0.15))
modelGl.add(Dense(units=2))
modelGl.add(Activation('softmax'))
modelGl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [43]:
callback = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True
)
history = modelGl.fit(X_train, Y_train, batch_size=1000, epochs=50, validation_data=(X_vald, Y_vald), callbacks=[callback])

Train on 66517 samples, validate on 14254 samples
Epoch 1/50
66517/66517 [==============================] - 17s 262us/step - loss: 0.6686 - acc: 0.6215 - val_loss: 0.6464 - val_acc: 0.6383
Epoch 2/50
66517/66517 [==============================] - 16s 248us/step - loss: 0.6490 - acc: 0.6382 - val_loss: 0.6458 - val_acc: 0.6404
Epoch 3/50
66517/66517 [==============================] - 17s 249us/step - loss: 0.6464 - acc: 0.6385 - val_loss: 0.6438 - val_acc: 0.6404
Epoch 4/50
66517/66517 [==============================] - 17s 251us/step - loss: 0.6420 - acc: 0.6405 - val_loss: 0.6418 - val_acc: 0.6426
Epoch 5/50
66517/66517 [==============================] - 16s 245us/step - loss: 0.6367 - acc: 0.6448 - val_loss: 0.6395 - val_acc: 0.6390
Epoch 6/50
66517/66517 [==============================] - 16s 246us/step - loss: 0.6330 - acc: 0.6462 - val_loss: 0.6275 - val_acc: 0.6412
Epoch 7/50
66517/66517 [==============================] - 16s 246us/step - loss: 0.6212 - acc: 0.6536 - val_loss: 0.

In [44]:
y_pred = modelGl.predict_classes(X_test)

In [45]:
y_test = ( Y_test[:,1] > 0 ) * 1

In [46]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
print(confusion_matrix(y_test, y_pred))

[[ 6361  4089]
 [ 2454 15604]]


In [48]:
print(f"Precision: {precision_score(y_test, y_pred)}\nRecall:    {recall_score(y_test,y_pred)}\nF1:        {f1_score(y_test, y_pred)}\nAccuracy:  {accuracy_score(y_test, y_pred)}")

Precision: 0.7923627684964201
Recall:    0.864104551999114
F1:        0.8266800879446903
Accuracy:  0.7704854777606286


<b> This is a poor model. However I will save the components</b><br/>
This might be becasue the char embeddings have no notion of proper name contexts with genders while training<br/>
An alternative would be to train corpus with gender and proper name components for the letter embeddings

In [49]:
modelGl.save("assets\models\LSTMGloveAuto")

In [50]:
encoder.save("assets\models\AutoEncoder")

In [55]:
import joblib
joblib.dump(scaler, "assets\models\Minmaxscaler")

['assets\\models\\Minmaxscaler']